## On calcule les prix approximatifs de 2021 avec l'inflation à l'aide d'un fichier de l'INSEE retravaillé au préalable

In [63]:
import pandas as pd

In [64]:
X = pd.read_csv("../traitement_photos/X_avec_prix_et_images.csv")
X["nearly_price"]=X["PrixMoyen_M2"]*X["size"]

In [65]:
X["departement"]=X["postal_code"]
X["departement"][X["departement"] < 10000]=X["departement"].astype(str).str[:1].astype(int)
X["departement"][X["departement"] >= 10000]=X["departement"].astype(str).str[:2].astype(int)

C:\Users\maila\AppData\Local\Temp\ipykernel_21592\2441469366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["departement"][X["departement"] < 10000]=X["departement"].astype(str).str[:1].astype(int)
C:\Users\maila\AppData\Local\Temp\ipykernel_21592\2441469366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["departement"][X["departement"] >= 10000]=X["departement"].astype(str).str[:2].astype(int)


> Pour les maisons et les appartements :

In [66]:
X["nearly_price_inflation"]=None

In [67]:

departements_dataset = X["departement"].unique().tolist()
# departements présents explicitement dans la bdd de l'INSEE
departements = [75,77,78,91,92,93,94,95,4,5,6,13,83,84,1,3,7,15,26,38,42,43,63,69,73,74,2,59,60,62,80]
taux_inflation_apparts = [1.034802784,1.067698259,1.068328717,1.070949185,1.08037225,1.094952951,1.089005236,1.074144487,1.066918002,1.066918002,1.066918002,1.066918002,1.066918002,1.066918002,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.114111498,1.110377358,1.110377358,1.110377358,1.110377358,1.110377358]
taux_inflation_maison = [1.034802784,1.092380952,1.075418994,1.080449018,1.097797357,1.094977169,1.099188458,1.086997194,1.083409716,1.083409716,1.083409716,1.083409716,1.083409716,1.083409716,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.095829636,1.088180113,1.088180113,1.088180113,1.088180113,1.088180113]

# On remplit les valeurs pour les maisons et les apparts dont on a les numéros
for i in range(len(departements)):
    X["nearly_price_inflation"][(X["property_type"] == "appartement") & (X["departement"] == departements[i])] = X["nearly_price"][(X["property_type"] == "appartement") & (X["departement"] == departements[i])] * taux_inflation_apparts[i]
    X["nearly_price_inflation"][(X["property_type"] == "maison") & (X["departement"] == departements[i])] = X["nearly_price"][(X["property_type"] == "maison") & (X["departement"] == departements[i])] * taux_inflation_maison[i]

province_apparts = 1.106019766
province_maisons = 1.101083032

departements_not_in_INSEE = [dep for dep in departements_dataset if dep not in departements]

for dep in departements_not_in_INSEE:
# on remplit les valeurs pour les maisons et les apparts dont on a pas les numéros (donc "Province")
    X["nearly_price_inflation"][(X["property_type"] == "appartement") & (X["departement"] == dep)] = X["nearly_price"][(X["property_type"] == "appartement") & (X["departement"] == dep)] * province_apparts
    X["nearly_price_inflation"][(X["property_type"] == "maison") & (X["departement"] == dep)] = X["nearly_price"][(X["property_type"] == "maison") & (X["departement"] == dep)] * province_maisons

# X["nearly_price_inflation"][X["property_type"] == "appartement" and "departement" == 75] = X["nearly_price"][X["property_type"] == "appartement"] * 125.3/115.6
# X["nearly_price_inflation"][X["property_type"] == "maison"] = X["nearly_price"][X["property_type"] == "maison"] * 121.3/110.4

C:\Users\maila\AppData\Local\Temp\ipykernel_21592\4291456083.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["nearly_price_inflation"][(X["property_type"] == "appartement") & (X["departement"] == departements[i])] = X["nearly_price"][(X["property_type"] == "appartement") & (X["departement"] == departements[i])] * taux_inflation_apparts[i]
C:\Users\maila\AppData\Local\Temp\ipykernel_21592\4291456083.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["nearly_price_inflation"][(X["property_type"] == "maison") & (X["departement"] == departements[i])] = X["nearly_price"][(X["property_type"] == "maison") & (X["depa

> Pour le reste des types, on remplit par la valeur moyenne du département :

In [68]:
X["nearly_price_inflation"] = X.groupby("departement")["nearly_price_inflation"].transform(lambda x: x.fillna(x.mean()))

In [69]:
X[["nearly_price_inflation", "nearly_price"]]

,nearly_price_inflation,nearly_price
0,3.478419e+05,3.260250e+05
1,2.453608e+05,2.202300e+05
2,7.824847e+04,7.106500e+04
3,3.322518e+05,3.017500e+05
4,4.251093e+05,3.815680e+05
...,...,...
46702,3.884912e+06,4.965120e+05
46703,2.589138e+05,2.426745e+05
46704,2.602907e+05,2.353400e+05
46705,3.077307e+05,2.794800e+05


In [70]:
X = X.drop(columns=["departement"])
X.to_csv("../Prix_metre/X_avec_prix_et_images_et_inflation.csv", index=False)